<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
import nltk
from bs4 import BeautifulSoup

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import ebooklib
from ebooklib import epub
from glob import glob
sources = list(glob('data/*.epub'))

def chapter_to_paragraphs(chapter):
    soup = BeautifulSoup(chapter.get_body_content(), 'html.parser')
    text = [para.get_text() for para in soup.find_all('p')]
    return text

#sources

[nltk_data] Downloading package punkt to /home/carl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/carl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/carl/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/carl/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
test_source = 'data/24_fifth_elephant.epub'

In [3]:
# Parameters
test_source = "data/41_the_shepherds_crown.epub"


In [4]:
from pathlib import Path
base_title = Path(test_source).stem
book = epub.read_epub(test_source)
chapters = {}
for doc in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
    print(doc.get_name())
    if 'part' in doc.get_name():
        chapters[doc.get_name()] = chapter_to_paragraphs(doc)
        continue
    # res = nltk.word_tokenize(str(doc.content))
    # text += " ".join([r for r in res if r.isalnum()])
print(chapters.keys())

OPS/xhtml/about_author.xhtml
OPS/xhtml/about_book.xhtml
OPS/xhtml/acknowledgements.xhtml
OPS/xhtml/afeegle.xhtml
OPS/xhtml/afterword.xhtml
OPS/xhtml/also_by.xhtml
OPS/xhtml/back_ad.xhtml
OPS/xhtml/chapter_001.xhtml
OPS/xhtml/chapter_002.xhtml
OPS/xhtml/chapter_003.xhtml
OPS/xhtml/chapter_004.xhtml
OPS/xhtml/chapter_005.xhtml
OPS/xhtml/chapter_006.xhtml
OPS/xhtml/chapter_007.xhtml
OPS/xhtml/chapter_008.xhtml
OPS/xhtml/chapter_009.xhtml
OPS/xhtml/chapter_010.xhtml
OPS/xhtml/chapter_011.xhtml
OPS/xhtml/chapter_012.xhtml
OPS/xhtml/chapter_013.xhtml
OPS/xhtml/chapter_014.xhtml
OPS/xhtml/chapter_015.xhtml
OPS/xhtml/chapter_016.xhtml
OPS/xhtml/chapter_017.xhtml
OPS/xhtml/chapter_018.xhtml
OPS/xhtml/chapter_019.xhtml
OPS/xhtml/contents.xhtml
OPS/xhtml/copyright.xhtml
OPS/xhtml/cover.xhtml
OPS/xhtml/dedication.xhtml
OPS/xhtml/epilogue_split_000.xhtml
OPS/xhtml/epilogue_split_001.xhtml
OPS/xhtml/praise.xhtml
OPS/xhtml/prologue.xhtml
OPS/xhtml/sneak_preview.xhtml
OPS/xhtml/title.xhtml
contents_u1

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
import nltk

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_names(text):
    nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
    for nltk_result in nltk_results:
        if type(nltk_result) == Tree:
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            print ('Type: ', nltk_result.label(), 'Name: ', name)
            
def get_persons(text):
    people = set()
    nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
    for nltk_result in nltk_results:
        if type(nltk_result) == Tree:
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            if nltk_result.label() == 'PERSON':
                people.add(name)
    return people
            
            
sample_paragraphs = list(chapters.values())[0]




for s in sample_paragraphs[-10:]:
    print('\n\n\n',s,'\n')
    #print(sample_text)

    get_names(s)
    print(get_persons(s))

IndexError: list index out of range

In [ ]:
display([len(l) for l in chapters.values()])
from collections import Counter
from itertools import combinations

def process_chapter(chapter):

    people_counter = Counter()
    combo_counter = Counter()

    def get_connections(paragraph):
        people = get_persons(paragraph)
        if not people: return
        for p in people:
            people_counter.update([p])

        for combo in combinations(people,2):
            combo_counter.update([combo])


    for p in chapter:
        # print(p)
        get_connections(p)
    return people_counter.copy(),combo_counter.copy()
    
people_counter_chap1, combo_counter_chap1 = process_chapter(sample_paragraphs)
print('People counter:', people_counter_chap1)
print('Combo counter:' ,combo_counter_chap1)

In [ ]:
from tqdm import tqdm
people_counters = {}
combo_counters = {}
pb = tqdm(chapters.items())
for chapter_name, paragraphs in pb:
    people_counters[chapter_name], combo_counters[chapter_name] = process_chapter(paragraphs)
    

In [ ]:

outdir = Path('data') /'processed' / base_title
if  not outdir.exists(): outdir.mkdir(parents=True)
print(outdir.exists())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt



df = pd.DataFrame(people_counters.values(), (f'Chapter {i}' for i, _ in enumerate(people_counters.keys()))).fillna(0)
select_cols = df.sum(axis=0)>10
filtered_people = df.loc[:,select_cols].copy()

df.to_csv(outdir/f"{base_title}_people.csv")
#filtered_people.plot()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(combo_counters.values(), (f'Chapter {i}' for i, _ in enumerate(combo_counters.keys()))).fillna(0)
df.to_csv(outdir/f"{base_title}_combos.csv")
display(df.head())
select_cols = df.sum(axis=0)>3
filtered_combos = df.loc[:,select_cols].copy()
#filtered_combos.plot()

In [ ]:
import networkx as nx
from itertools import chain

g = nx.Graph()

people_from_combos = set(chain(*[[*c] for c in filtered_combos.columns]))
print(people_from_combos)
g.add_nodes_from([(p,{'label':p}) for p in people_from_combos])

summed_weights = filtered_combos.sum(axis=0)

print(summed_weights)

edges_with_weights = [(*edge,weight) for edge, weight  in zip(summed_weights.index, summed_weights.values)]
print(edges_with_weights)


g.add_weighted_edges_from(edges_with_weights)




In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot()

nx.draw(g,ax=ax, pos=nx.spring_layout(g))
nx.draw_networkx_labels(g,pos=nx.spring_layout(g),ax=ax)
plt.show()








nx.write_gexf(g, outdir/f"{base_title}.gexf")
